In [55]:
from openai import OpenAI
import pandas as pd
import pickle
import openai
from tqdm import tqdm  # For progress bar, useful for large datasets

PICKLE_PATH     = 'Saved/textdata.pkl'
API_KEY         = open("key.txt", "r").read().strip("\n")
EMBEDDING_MODEL = "text-embedding-3-small"
SAVE_EMBDEDED_PKL   = f'Saved/textembed ({EMBEDDING_MODEL}).pkl'
SAVE_EMBDEDED_CSV   = f'Saved/textembed ({EMBEDDING_MODEL}).csv'


In [56]:
def save_data(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)
    print(f"Data saved to {filename}")

def load_data(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)
 
data = load_data(PICKLE_PATH)
df = pd.DataFrame(data, columns=['Document', 'Page', 'Block', "WordCount", 'Text'])
#df = df[df['Document'] == "Ustawa.pdf"]
#df = df.head(5)


In [57]:
df

Document  Page  Block  WordCount  \
0   DDDD & DD.pdf     1      1         24   
1   DDDD & DD.pdf     2      1        125   
2   DDDD & DD.pdf     2      2         25   
3   DDDD & DD.pdf     2      3         54   
4   DDDD & DD.pdf     3      1        119   
..            ...   ...    ...        ...   
67     Ustawa.pdf    14      1         27   
68     Ustawa.pdf    14      2         54   
69     Ustawa.pdf    14      3         74   
70     Ustawa.pdf    14      4         78   
71     Ustawa.pdf    15      1         42   

                                                 Text  
0   DDDD & DD. Dynamicznie Dedukowane Drzewa Decyz...  
1   W branży jubilerskiej, gdzie decyzje o zarządz...  
2   omówienia, za przykład weźmy wybór następujący...  
3   - Poziom 3 ‘Kamienie’ np. Cyrkonia, Brylant, G...  
4   Python W zależności od potrzeb obliczone już w...  
..                                                ...  
67  ©Kancelaria Sejmu s. 14/15 2023-09-21 w art 18...  
68  umożliwiających spełnienie wymogów przewidzian...  
69  przewidzianym w art. 18 ust 1, w braku odmienn...  
70  umożliwiającą najpóźniej od dnia 1 październik...  
71  ©Kancelaria Sejmu s. 15/15 2023-09-21 Art 41 U...  

[72 rows x 5 columns]

In [58]:
from openai import OpenAI
client = OpenAI(api_key=API_KEY)

def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model=EMBEDDING_MODEL
    )
    # Assuming the response structure fits this format; adjust according to actual API response
    return response.data[0].embedding

embeddings = []

# Loop through each row in your filtered DataFrame to get embeddings
for text in tqdm(df['Text'], desc="Generating embeddings"):
    embeddings.append(get_embedding(text))

# Add the embeddings as a new column to your DataFrame
df['Embedding'] = embeddings

Generating embeddings: 100%|██████████| 72/72 [00:22<00:00,  3.19it/s]


In [59]:
save_data(df, SAVE_EMBDEDED_PKL)
with open(SAVE_EMBDEDED_CSV, mode='w', newline='', encoding='utf-8-sig') as file:
    df.to_csv(file, index=False)

Data saved to Saved/textembed (text-embedding-3-small).pkl


In [60]:
df

Document  Page  Block  WordCount  \
0   DDDD & DD.pdf     1      1         24   
1   DDDD & DD.pdf     2      1        125   
2   DDDD & DD.pdf     2      2         25   
3   DDDD & DD.pdf     2      3         54   
4   DDDD & DD.pdf     3      1        119   
..            ...   ...    ...        ...   
67     Ustawa.pdf    14      1         27   
68     Ustawa.pdf    14      2         54   
69     Ustawa.pdf    14      3         74   
70     Ustawa.pdf    14      4         78   
71     Ustawa.pdf    15      1         42   

                                                 Text  \
0   DDDD & DD. Dynamicznie Dedukowane Drzewa Decyz...   
1   W branży jubilerskiej, gdzie decyzje o zarządz...   
2   omówienia, za przykład weźmy wybór następujący...   
3   - Poziom 3 ‘Kamienie’ np. Cyrkonia, Brylant, G...   
4   Python W zależności od potrzeb obliczone już w...   
..                                                ...   
67  ©Kancelaria Sejmu s. 14/15 2023-09-21 w art 18...   
68  umożliwiających spełnienie wymogów przewidzian...   
69  przewidzianym w art. 18 ust 1, w braku odmienn...   
70  umożliwiającą najpóźniej od dnia 1 październik...   
71  ©Kancelaria Sejmu s. 15/15 2023-09-21 Art 41 U...   

                                            Embedding  
0   [-0.011432180181145668, 0.029889998957514763, ...  
1   [-0.06011172756552696, 0.020597895607352257, 0...  
2   [-0.0008431045571342111, 0.057511720806360245,...  
3   [-0.05206809565424919, 0.045071445405483246, 0...  
4   [-0.0054521868005394936, 0.04055621474981308, ...  
..                                                ...  
67  [-0.04066246375441551, 0.08031929284334183, 0....  
68  [0.03570476546883583, 0.051144663244485855, 0....  
69  [-0.02101951092481613, 0.09580406546592712, 0....  
70  [0.029946209862828255, 0.05419602245092392, 0....  
71  [-0.03706207126379013, 0.03974829614162445, -0...  

[72 rows x 6 columns]